# Habitat Suitability Analysis

In [ ]:
import os
import zipfile

# Path to target folder
folder = "/content/bio_oracle/bio_oracle"

# 🧭 Step 1: Create the directory if it doesn't exist
if not os.path.exists(folder):
    os.makedirs(folder, exist_ok=True)
    print(f"✅ Created folder: {folder}")
else:
    print(f"📁 Folder already exists: {folder}")

# 🧭 Step 2: Make sure your Bio-Oracle ZIP file is available in /content
biooracle_zip = "/content/bio_oracle.zip"

if not os.path.exists(biooracle_zip):
    print(f"⚠️ Missing Bio-Oracle archive: {biooracle_zip}")
    print("➡️ Please upload 'bio_oracle.zip' to your Colab environment first (using the left file panel).")
else:
    # 🧭 Step 3: Extract the main Bio-Oracle zip if not already extracted
    with zipfile.ZipFile(biooracle_zip, 'r') as zip_ref:
        zip_ref.extractall(folder)
    print("✅ Main Bio-Oracle zip extracted successfully!")

    # 🧭 Step 4: Extract any internal .zip files inside the folder
    for file in os.listdir(folder):
        if file.endswith(".zip"):
            path = os.path.join(folder, file)
            with zipfile.ZipFile(path, 'r') as zip_ref:
                zip_ref.extractall(folder)
            print("📦 Extracted internal archive:", file)


✅ Created folder: /content/bio_oracle/bio_oracle
✅ Main Bio-Oracle zip extracted successfully!


****Step 1 — Data Collection****

Gathered here are 2 key datasets:

Species occurrence data

Source: GBIF (Global Biodiversity Information Facility)

What it contains: Geographic coordinates where Rugulopteryx okamurae has been observed.

Each record = presence point (where the species exists).

Environmental variables

Source: Bio-Oracle marine environmental rasters

Variables: temperature, salinity, chlorophyll, oxygen, productivity, and pH

Each raster provides global environmental conditions at the sea surface.

In [ ]:
!wget https://naturalearth.s3.amazonaws.com/110m_cultural/ne_110m_admin_0_countries.zip -O ne_110m_admin_0_countries.zip
!unzip -o ne_110m_admin_0_countries.zip -d naturalearth

--2025-11-06 14:50:06--  https://naturalearth.s3.amazonaws.com/110m_cultural/ne_110m_admin_0_countries.zip
Resolving naturalearth.s3.amazonaws.com (naturalearth.s3.amazonaws.com)... 52.92.207.41, 52.92.207.33, 52.92.130.153, ...
Connecting to naturalearth.s3.amazonaws.com (naturalearth.s3.amazonaws.com)|52.92.207.41|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 214976 (210K) [application/zip]
Saving to: ‘ne_110m_admin_0_countries.zip’

ne_110m_admin_0_cou 100%[===================>] 209.94K  1000KB/s    in 0.2s    

2025-11-06 14:50:06 (1000 KB/s) - ‘ne_110m_admin_0_countries.zip’ saved [214976/214976]

Archive:  ne_110m_admin_0_countries.zip
  inflating: naturalearth/ne_110m_admin_0_countries.README.html  
 extracting: naturalearth/ne_110m_admin_0_countries.VERSION.txt  
 extracting: naturalearth/ne_110m_admin_0_countries.cpg  
  inflating: naturalearth/ne_110m_admin_0_countries.dbf  
  inflating: naturalearth/ne_110m_admin_0_countries.prj  
  inflating: nat

In [ ]:
import geopandas as gpd
world = gpd.read_file("naturalearth/ne_110m_admin_0_countries.shp")

In [ ]:
import numpy as np

def filter_points_within_raster(gdf, raster_path):
    """Filter GBIF points to those that fall within a raster's extent."""
    with rasterio.open(raster_path) as src:
        minx, miny, maxx, maxy = src.bounds
        gdf_filtered = gdf.cx[minx:maxx, miny:maxy]
    return gdf_filtered


In [ ]:
import os

# Path to the folder where all your BioOracle rasters are stored
biooracle_dir = "/content/bio_oracle/bio_oracle"

# List of raster files you want to use (update the names if needed)
rasters = {
    "SST_mean": "BO_sstmean.tif",
    "SST_range": "BO_sstrange.tif",
    "Salinity": "BO_salinity.tif",
    "Chlorophyll": "BO_chlomean.tif"
}


In [ ]:
for var, filename in rasters.items():
    raster_path = os.path.join(biooracle_dir, filename)
    if os.path.exists(raster_path):
        try:
            gdf_in = filter_points_within_raster(gdf, raster_path)
            with rasterio.open(raster_path) as src:
                coords = [(x, y) for x, y in zip(gdf_in.geometry.x, gdf_in.geometry.y)]
                values = []
                for val in src.sample(coords):
                    v = val[0]
                    if v is not None and not np.isnan(v):
                        values.append(float(v))
                    else:
                        values.append(None)

            # Align back to gdf size (NaN for filtered-out points)
            extracted_data[var] = [None] * len(gdf)
            mask = gdf.index.isin(gdf_in.index)
            for i, v in zip(gdf[mask].index, values):
                extracted_data[var][i] = v

            print(f"✅ Extracted {sum(pd.notna(values))} valid values for {var}")
            valid_rasters_present = True

        except Exception as e:
            print(f"❌ Error reading raster {filename}: {e}")
            extracted_data[var] = [None] * len(gdf)
    else:
        print(f"⚠️ Missing raster: {filename}")
        extracted_data[var] = [None] * len(gdf)


In [ ]:
# -------------------------------------------------------------
# 🌊 VALIDATE COORDINATES WITHIN RASTER EXTENT AND OCEAN MASK
# -------------------------------------------------------------
import numpy as np
import rasterio
from rasterio.features import geometry_mask

# Load one reference raster (e.g., temperature) to check coverage
reference_raster = os.path.join(biooracle_dir, "Present.Surface.Temperature.Mean.tif")
if os.path.exists(reference_raster):
    with rasterio.open(reference_raster) as src:
        raster_bounds = src.bounds
        print("🗺️ Raster coverage bounds:", raster_bounds)

        # Filter GBIF points within raster extent
        gdf = gdf[
            (gdf.geometry.x >= raster_bounds.left) &
            (gdf.geometry.x <= raster_bounds.right) &
            (gdf.geometry.y >= raster_bounds.bottom) &
            (gdf.geometry.y <= raster_bounds.top)
        ]

        print(f"✅ Retained {len(gdf)} GBIF points within raster coverage")

        # Optional: derive ocean mask (non-land pixels)
        # Any NaN pixel in raster is considered land
        coords = [(x, y) for x, y in zip(gdf.geometry.x, gdf.geometry.y)]
        values = [v[0] for v in src.sample(coords)]
        mask = np.array([not np.isnan(v) for v in values])

        gdf = gdf.loc[mask]
        print(f"🌊 Retained {len(gdf)} marine GBIF points (inside Bio-Oracle ocean areas)")
else:
    print("⚠️ Reference raster not found. Skipping spatial validation.")


In [ ]:
# =============================================================
# 🌿 Early Detection Data Extraction for Rugulopteryx okamurae
# =============================================================

# Install dependencies (run once in your Colab)
!apt-get install -y libgeos-dev
!pip install rasterio geopandas seaborn cartopy

import os
import zipfile
import rasterio
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from shapely.geometry import Point
import requests

# -------------------------------------------------------------
# 1️⃣  UNZIP BIO-ORACLE DATA
# -------------------------------------------------------------
biooracle_zip = "bio_oracle.zip"
biooracle_dir = "./bio_oracle/bio_oracle"

if not os.path.exists(biooracle_dir):
    os.makedirs(biooracle_dir, exist_ok=True)
    if os.path.exists("/content/" + biooracle_zip):
        with zipfile.ZipFile("/content/" + biooracle_zip, "r") as zip_ref:
            zip_ref.extractall(biooracle_dir)
        print("✅ Bio-Oracle files extracted!")
    else:
        print(f"⚠️ {biooracle_zip} not found in /content. Please upload it.")
else:
    print("✅ Bio-Oracle already extracted.")

# -------------------------------------------------------------
# 2️⃣  LOAD GBIF CSV DATA
# -------------------------------------------------------------
gbif_file = '/content/gbif_0013072-251025141854904.csv'
if not os.path.exists(gbif_file):
    gbif_file = '/content/gbif_0013072-251025141854904 (2).csv'  # alternate name if needed

if os.path.exists(gbif_file):
    df = pd.read_csv(gbif_file, sep='\t')

    # Filter valid coords
    df = df.dropna(subset=["decimalLatitude", "decimalLongitude"])
    df = df[(df["decimalLatitude"].between(-90, 90)) & (df["decimalLongitude"].between(-180, 180))]

    print(f"Loaded {len(df)} Rugulopteryx okamurae records")

    # Convert to GeoDataFrame
    gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.decimalLongitude, df.decimalLatitude), crs="EPSG:4326")
else:
    print(f"⚠️ GBIF file not found: {gbif_file}. Please upload the GBIF data.")
    gdf = gpd.GeoDataFrame(columns=['decimalLatitude', 'decimalLongitude', 'geometry'], crs="EPSG:4326")

# -------------------------------------------------------------
# -------------------------------------------------------------
# 3️⃣  LOAD RASTERS AND EXTRACT VALUES
# -------------------------------------------------------------
rasters = {
    "temperature": "Present.Surface.Temperature.Mean.tif",
    "salinity": "Present.Surface.Salinity.Mean.tif",
    "chlorophyll": "Present.Surface.Chlorophyll.Mean.tif",
    "oxygen": "Present.Surface.Dissolved.oxygen.Mean.tif",
    "productivity": "Present.Surface.Primary.productivity.Mean.tif",
    "ph": "Present.Surface.pH.BOv2_2.tif",
}

extracted_data = {}
valid_rasters_present = False

for var, filename in rasters.items():
    raster_path = os.path.join(biooracle_dir, filename)
    if os.path.exists(raster_path):
        try:
            with rasterio.open(raster_path) as src:
                coords = [(x, y) for x, y in zip(gdf.geometry.x, gdf.geometry.y)]
                values = [v[0] for v in src.sample(coords)]
                extracted_data[var] = values
            print(f"✅ Extracted values for {var}")
            valid_rasters_present = True
        except Exception as e:
            print(f"❌ Error reading raster {filename}: {e}")
            extracted_data[var] = [None] * len(gdf)
    else:
        print(f"⚠️ Missing raster: {filename}")
        extracted_data[var] = [None] * len(gdf)

# -------------------------------------------------------------
# 4️⃣  MERGE RESULTS AND CLEAN BIO-ORACLE VALUES
# -------------------------------------------------------------
if valid_rasters_present:
    env_df = pd.DataFrame(extracted_data)
    df_reset = df.reset_index(drop=True)
    env_df_reset = env_df.reset_index(drop=True)
    combined_df = pd.concat([df_reset, env_df_reset], axis=1)

    # 🧹 Clean invalid raster fill values
    import numpy as np
    fill_values = [-1.7e308, -3.4e38]
    for col in env_df.columns:
        combined_df[col] = combined_df[col].replace(fill_values, np.nan)

    # Drop rows where all environmental values are NaN
    combined_df = combined_df.dropna(subset=env_df.columns, how="all")

    print(f"\n✅ Combined dataset shape after cleaning: {combined_df.shape}")
    print(combined_df[env_df.columns].describe())
else:
    print("\n⚠️ No valid rasters found. Cannot create combined dataset.")
    combined_df = df.copy()

# -------------------------------------------------------------
# 5️⃣  CORRELATION ANALYSIS
# -------------------------------------------------------------
if valid_rasters_present and not combined_df.empty:
    try:
        env_columns_present = [col for col in env_df.columns if col in combined_df.columns]
        # Remove constant columns (no variance)
        variable_cols = [col for col in env_columns_present if combined_df[col].nunique(dropna=True) > 1]

        if variable_cols:
            corr_matrix = combined_df[variable_cols].corr()
            plt.figure(figsize=(7,6))
            sns.heatmap(corr_matrix, annot=True, cmap="coolwarm", fmt=".2f")
            plt.title("Correlation between Bio-Oracle environmental variables")
            plt.show()
        else:
            print("\n⚠️ No environmental columns with variable data for correlation.")
    except Exception as e:
        print(f"\n❌ Error during correlation analysis: {e}")
else:
    print("\n⚠️ Skipping correlation analysis as environmental data is not available.")

# -------------------------------------------------------------
# 6️⃣  VISUALIZE OCCURRENCES ON MAP WITH CARTOPY
# -------------------------------------------------------------
if not gdf.empty:
    try:
        fig = plt.figure(figsize=(12,8))
        ax = plt.axes(projection=ccrs.PlateCarree())

        # Natural Earth features
        ax.add_feature(cfeature.LAND, facecolor='lightgrey')
        ax.add_feature(cfeature.OCEAN, facecolor='lightblue')
        ax.add_feature(cfeature.COASTLINE)
        ax.add_feature(cfeature.BORDERS, linestyle=':')

        # Dynamic extent with buffer
        buffer = 5  # degrees
        minx, miny, maxx, maxy = gdf.total_bounds
        ax.set_extent([minx - buffer, maxx + buffer, miny - buffer, maxy + buffer], crs=ccrs.PlateCarree())

        # Plot occurrences
        ax.scatter(gdf.geometry.x, gdf.geometry.y,
                   color='darkred', s=20, alpha=0.7,
                   transform=ccrs.PlateCarree(),
                   label='Rugulopteryx okamurae')

        plt.title("🌊 Rugulopteryx okamurae Occurrences (GBIF)")
        plt.legend()
        plt.show()

    except Exception as e:
        print(f"\n❌ Error visualizing occurrences on map with Cartopy: {e}")
else:
    print("\n⚠️ Skipping map visualization as no valid GBIF data was loaded.")

# -------------------------------------------------------------
# 7️⃣  SAVE RESULTS
# -------------------------------------------------------------
if not combined_df.empty:
    try:
        combined_df.to_csv("Rugulopteryx_environmental_data.csv", index=False)
        print("\n✅ Saved final dataset as 'Rugulopteryx_environmental_data.csv'")
    except Exception as e:
        print(f"\n❌ Error saving combined dataset: {e}")
else:
    print("\n⚠️ Skipping saving combined dataset as it is empty.")


In [ ]:
# -------------------------------------------------------------
# 5️⃣b  CORRELATION WITH LATITUDE, LONGITUDE & PRESENCE
# -------------------------------------------------------------

# Add presence column (all ones for current GBIF data)
# If later you include absence/background points, you can set 0 for absences.
combined_df["presence"] = 1

# Select variables for correlation
geo_env_cols = ["decimalLatitude", "decimalLongitude", "presence",
                "temperature", "salinity", "chlorophyll", "oxygen", "productivity", "ph"]

# Keep only columns that actually exist
geo_env_cols = [col for col in geo_env_cols if col in combined_df.columns]

# Drop rows with NaN
geo_env_data = combined_df[geo_env_cols].dropna()

if len(geo_env_data) > 1:
    corr_geo_env = geo_env_data.corr()

    plt.figure(figsize=(8, 6))
    sns.heatmap(corr_geo_env, annot=True, cmap="coolwarm", fmt=".2f")
    plt.title("Correlation between Latitude, Longitude, Presence and Bio-Oracle Variables")
    plt.show()
else:
    print("⚠️ Not enough valid data to compute extended correlation matrix.")


****⚙️ Step 2 — Generating Pseudo-Absences****

Since you only had presence data, you added pseudo-absence points randomly distributed across the ocean (500 in your case).

Why?
To contrast conditions where the species is known not to occur (or is unlikely).
This is standard in SDM when absence data is unavailable.

In [ ]:
import numpy as np
from shapely.geometry import Point

# Create 500 pseudo-absence points randomly across raster extent
with rasterio.open(os.path.join(biooracle_dir, "Present.Surface.Temperature.Mean.tif")) as src:
    minx, miny, maxx, maxy = src.bounds
    np.random.seed(42)
    rand_lon = np.random.uniform(minx, maxx, 500)
    rand_lat = np.random.uniform(miny, maxy, 500)
    abs_df = pd.DataFrame({"decimalLongitude": rand_lon, "decimalLatitude": rand_lat, "presence": 0})
    abs_gdf = gpd.GeoDataFrame(abs_df, geometry=gpd.points_from_xy(rand_lon, rand_lat), crs="EPSG:4326")

# Combine with presence data
presence_df = combined_df.copy()
presence_df["presence"] = 1
merged_data = pd.concat([presence_df, abs_gdf], ignore_index=True)
# -------------------------------------------------------------
# EXTRACT BIO-ORACLE VALUES FOR ALL POINTS (presence + absence)
# -------------------------------------------------------------
coords = list(zip(merged_data["decimalLongitude"], merged_data["decimalLatitude"]))

env_vars = ["temperature", "salinity", "chlorophyll", "oxygen", "productivity", "ph"]
raster_paths = {
    "temperature": os.path.join(biooracle_dir, "Present.Surface.Temperature.Mean.tif"),
    "salinity": os.path.join(biooracle_dir, "Present.Surface.Salinity.Mean.tif"),
    "chlorophyll": os.path.join(biooracle_dir, "Present.Surface.Chlorophyll.Mean.tif"),
    "oxygen": os.path.join(biooracle_dir, "Present.Surface.Dissolved.oxygen.Mean.tif"),
    "productivity": os.path.join(biooracle_dir, "Present.Surface.Primary.productivity.Mean.tif"),
    "ph": os.path.join(biooracle_dir, "Present.Surface.pH.BOv2_2.tif"),
}

for var in env_vars:
    with rasterio.open(raster_paths[var]) as src:
        merged_data[var] = [v[0] for v in src.sample(coords)]

# Clean fill values (Bio-Oracle missing data codes)
fill_values = [-1.7e308, -3.4e38]
merged_data.replace(fill_values, np.nan, inplace=True)

# Drop rows where all environmental variables are NaN
merged_data.dropna(subset=env_vars, how="all", inplace=True)

print(f"✅ Extracted environmental data for {len(merged_data)} presence/absence points")
corr_vars = ["decimalLatitude", "decimalLongitude", "presence"] + env_vars
corr_matrix = merged_data[corr_vars].corr()

plt.figure(figsize=(9,7))
sns.heatmap(corr_matrix, annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Correlation between Latitude, Longitude, Presence and Bio-Oracle Variables")
plt.show()
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

features = env_vars
merged_clean = merged_data.dropna(subset=features + ["presence"])
X = merged_clean[features]
y = merged_clean["presence"]

# Standardize predictors
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Fit logistic regression
model = LogisticRegression()
model.fit(X_scaled, y)

# Show variable importance
coeffs = pd.DataFrame({
    "Variable": features,
    "Coefficient": model.coef_[0]
}).sort_values("Coefficient", ascending=False)

print("\n🌿 Environmental influence on presence probability:")
print(coeffs)


****Step 3 — Statistical Model: Logistic Regression****

This is the model you used to predict suitability.

Why Logistic Regression?

It’s a classic, interpretable binary classification model.

It estimates the probability that a given environment is “suitable” (i.e., presence = 1).

It’s mathematically equivalent to the “GLM” (Generalized Linear Model) used in many ecological studies.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

# Define variables
env_vars = ["temperature", "salinity", "chlorophyll", "oxygen", "productivity", "ph"]

# Drop missing rows
merged_clean = merged_data.dropna(subset=env_vars + ["presence"])

X = merged_clean[env_vars]
y = merged_clean["presence"]

# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Fit logistic regression model
model = LogisticRegression()
model.fit(X_scaled, y)

# Predict probability of presence
merged_clean["suitability"] = model.predict_proba(X_scaled)[:, 1]

print("✅ Habitat suitability scores calculated!")
merged_clean[["decimalLongitude", "decimalLatitude", "presence", "suitability"]].head()
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# Create figure
fig = plt.figure(figsize=(14,8))
ax = plt.axes(projection=ccrs.PlateCarree())

# Add base map features
ax.add_feature(cfeature.LAND, facecolor='lightgrey')
ax.add_feature(cfeature.OCEAN, facecolor='lightblue')
ax.add_feature(cfeature.COASTLINE, linewidth=0.5)
ax.add_feature(cfeature.BORDERS, linestyle=':', linewidth=0.5)

# Add gridlines
ax.gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False)

# Plot suitable areas
high = merged_clean[merged_clean["suitability"] > 0.5]
low = merged_clean[merged_clean["suitability"] <= 0.5]

ax.scatter(low["decimalLongitude"], low["decimalLatitude"],
           color="red", s=20, alpha=0.5, label="Unsuitable (<0.5)", transform=ccrs.PlateCarree())
ax.scatter(high["decimalLongitude"], high["decimalLatitude"],
           color="green", s=25, alpha=0.7, label="Suitable (>0.5)", transform=ccrs.PlateCarree())

plt.title("🌊 Environmental Suitability for Rugulopteryx okamurae", fontsize=14)
plt.legend(loc="lower left")
plt.show()
plt.figure(figsize=(14,8))
ax = plt.axes(projection=ccrs.PlateCarree())

ax.add_feature(cfeature.LAND, facecolor='lightgrey')
ax.add_feature(cfeature.OCEAN, facecolor='lightblue')
ax.add_feature(cfeature.COASTLINE)

sc = ax.scatter(
    merged_clean["decimalLongitude"],
    merged_clean["decimalLatitude"],
    c=merged_clean["suitability"],
    cmap="viridis",
    s=25,
    alpha=0.8,
    transform=ccrs.PlateCarree()
)

plt.colorbar(sc, label="Suitability probability (0–1)", orientation="vertical")
plt.title("Predicted Habitat Suitability for Rugulopteryx okamurae")
plt.show()


****🌍 Step 6 — Ecological Interpretation****

The map highlights regions where the combination of environmental variables (temperature, salinity, etc.) are most similar to the conditions of known Rugulopteryx okamurae populations.

These areas are considered potentially suitable — meaning, if the algae were introduced there, conditions might allow it to establish.

Unsuitable areas represent environmental mismatches, where the algae likely cannot survive or reproduce effectively.

In [ ]:
# -------------------------------------------------------------
# ✅  MODEL EVALUATION: ACCURACY, CONFUSION MATRIX, ROC CURVE
# -------------------------------------------------------------
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score, roc_curve
import matplotlib.pyplot as plt

# Predict binary presence/absence (threshold 0.5)
y_pred = (model.predict_proba(X_scaled)[:, 1] >= 0.5).astype(int)
y_true = y

# 1️⃣ Accuracy
accuracy = accuracy_score(y_true, y_pred)
print(f"✅ Model accuracy: {accuracy:.2f}")

# 2️⃣ Confusion matrix
cm = confusion_matrix(y_true, y_pred)
print("\n📊 Confusion Matrix:")
print(cm)

# 3️⃣ Classification report (precision, recall, F1-score)
print("\n📋 Classification Report:")
print(classification_report(y_true, y_pred, digits=2))

# 4️⃣ ROC curve and AUC
y_proba = model.predict_proba(X_scaled)[:, 1]
auc = roc_auc_score(y_true, y_proba)
fpr, tpr, _ = roc_curve(y_true, y_proba)

plt.figure(figsize=(6,5))
plt.plot(fpr, tpr, label=f"AUC = {auc:.2f}", color='darkorange')
plt.plot([0, 1], [0, 1], linestyle='--', color='grey')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve for Habitat Suitability Model')
plt.legend()
plt.show()
